In [ ]:
'''
Created By Josh Stokley to create stress and strain movie profile of simulated RC Walls
The class provides-
variables:
 self.refFile ........................... Defined txt file with variables related to RC wall
 self.ip1filesig ........................ stress data from integration point 1 of each shell element
 self.ip2filesig ........................ stress data from integration point 2 of each shell element
 self.ip1fileeps ........................ strain data from integration point 1 of each shell element
 self.ip2fileeps ........................ strain data from integration point 2 of each shell element
 self.filepath .......................... pathway to folder for the specific wall, holds reference file, tcl file, and recorded data
 self.tclFile ........................... pathway to tcl file
 self.name .............................. unique ID of wall (usually authors name + wall name)
 self.xNodes ............................ location of each node in the x direction
 self.yNodes ............................ location of each node in the y direction
 self.maxEle ............................ total amount of shell elements
 self.nodes_along_width ................. total amount of nodes along width of wall
 self.disp .............................. array of displacement values for each node at the top of the wall
 self.num_itr ........................... total number of simulated indices after skipping (currently grabbing 100 points to save time)
 self.total ............................. total number of simulated indices
 self.skip .............................. how many points in simulation being pulled
 self.files ............................. array of stress and strain files
 
 
methods:
 def __init__(self, refFile,ip1filesig,ip2filesig,ip1fileeps,ip2fileeps) ... constructor: initializes structure with reference File and stress/strain data, imports filepath,tclFile and name from reference file and runs grabInfo() and finish() functions
 def grabInfo(self) ........................................................ locations of nodes, nodes along the width, and total amount of shell elements are found
 def getReact(self,filename) ............................................... forces in the x direction at each node along the base are imported
 def getDisp(self,filename) ................................................ displacements in the y direction at each node along the top are imported
 def dispForce(self) ....................................................... length of the simulated points is found and amount of points used in the movies are defined
 def getLocations(self) .................................................... Stress/strain values come from the integration points, not nodes so those locations are defined and applied
 def extract(self,sig) ..................................................... data is extracted based on profile being shown, ie stress in the y, shear stress, minimum principal strain, etc....
 def plotMovie(self) ....................................................... the widget is put together with all extracted data
 def finish(self) .......................................................... stress/strain data is sorted through and movie is plotted
'''



import pandas as pd
import numpy as np
import os
import math as m
from scipy import io
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import plotly.io as pio
pio.renderers.default = 'iframe'



class CreateMovies():
    def __init__(self, refFile,ip1filesig,ip2filesig,ip1fileeps,ip2fileeps):
        #super().__init__(authorIndex=authorIndex) 
        self.refFile = refFile       
        self.ip1filesig,self.ip2filesig,self.ip1fileeps,self.ip2fileeps = ip1filesig,ip2filesig,ip1fileeps,ip2fileeps
        with open(self.refFile, 'rt') as h:
            data = [line.split() for line in h]
            self.filepath = data[8][0]
            self.tclFile = data[9][0]
            self.name = data[7][0]
        self.grabInfo()
        self.finish()
        
    def grabInfo(self):
        self.xNodes = []
        self.yNodes = []
        with open(self.tclFile, 'rt') as h:
            data = [line.split() for line in h]
            found = False

            for ii in range(len(data)):

                if len(data[ii]) > 1  and data[ii][0] == 'node':
                    if len(self.xNodes)> 1 and (float(data[ii][2])) == self.xNodes[0] and found == False:
                        self.nodes_along_width = (int(data[ii-2][1]))
                        found = True

                    self.xNodes.append(float(data[ii][2]))
                    self.yNodes.append(float(data[ii][3]))

                if len(data[ii]) > 1  and data[ii][1] == 'ShellDKGQ':   
                    self.maxEle = (int(data[ii][2]))
            
       # print(self.yNodes)
    def getReact(self,filename):
        with open(filename, 'r') as h:
            data = [line.split() for line in h]
            x = []
            i = 0
            loc = 0
            for line in data:
                if len(data[i])<self.nodes_along_width:
                    break
                for ii in range(0,self.nodes_along_width):
                    loc += float(data[i][ii])


                loc = -loc
                x.append(loc)
                loc = 0
                i += 1
            
        return x


    def getDisp(self,filename):
        with open(filename, 'r') as h:
            data = [line.split() for line in h]
            x = []
            i = 0
            loc = 0
            for line in data:
                if len(data[i])<self.nodes_along_width:
                    break
                for ii in range(0,self.nodes_along_width):
                        loc += float(data[i][ii])

                loc = loc/self.nodes_along_width
                x.append(loc)
                loc = 0
                i += 1
            
        return x
    
    def dispForce(self):
        loaded = io.loadmat('WallData.mat')
        data = loaded['WallData']
        loc = data[0,0]["UniqueID"]
        exDisp = data[0,0]["ExperimentalData"][0,0][self.name][0,0]['disp']
        exForce = data[0,0]["ExperimentalData"][0,0][self.name][0,0]['force']
        
        self.disp = self.getDisp(self.filepath+'topDispxcyc.txt')
        
        disp = self.disp
        for ii in range(len(loc)):
            if self.name == loc[ii][0][0]:
                index = ii
                break
        h = float(data[0,0]["Geometry"][0,0]['h_v'][index,0])
        
        drift = []
        for ii in disp:
            drift.append(ii/(h)*100)
            
        disp = drift
        self.disp = drift
        #force = self.getReact(self.filepath+'baseReactxcyc.txt')
        
        #if len(disp) < len(force):
        #    force = force[:len(disp)]
        #elif len(disp) > len(force):
        #    disp = disp[:len(force)]
        
        self.num_itr = len(disp)
        self.total = self.num_itr
        self.skip = 20
        if self.num_itr<10000:
            self.skip = 20
            self.num_itr =int(self.num_itr/self.skip)
        else:
            locnum = self.num_itr
            self.num_itr = int(500)
            self.skip = int(locnum/self.num_itr)
               
    def getLocations(self):

        x = []
        y = []
        for ii in range(self.nodes_along_width):
            x.append(self.xNodes[ii])


        length = (x[1] - x[0])/2
        print(length)
        x_loc = []
        for ii in range(len(x)):
            if x[ii] == 0:
                x_loc.append(length*1/3)
            elif x[ii] == x[-1]:
                x_loc.append(x[-1] - (length*2/3))
            else:
                x_loc.append(x[ii] - length*1/3)
                x_loc.append(x[ii] + length*1/3)

        y.append(0)
        for ii in range(len(self.yNodes)):
            if self.yNodes[ii] not in y:
                y.append(self.yNodes[ii])
            
        y_loc = []
        for ii in range(len(y)):
            if y[ii] == 0:
                y_loc.append(round(length - 1/(3)**(1/2),2)*length)
            elif y[ii] == y[-1]:
                break
            else:
                y_loc.append(round(y[ii] + ((length - 1/(3)**(1/2))*length),2))                
        

        print(y_loc)
        return x_loc,y_loc
    

    def extract(self,sig):
        if sig == True:
            file1 = self.files[0]
            file2 = self.files[1]
        else:
            file1 = self.files[2]
            file2 = self.files[3]                

        sigx = []
        sigy = []
        tau = []
        minP = []
        maxP = []
        for jj in range(len(file1)):
            y_stress = []
            x_stress = []
            tau_stress = []
            minP_stress = []
            maxP_stress = []
            z = 0
            locy = []
            locx = []
            loct = []
            locmin = []
            locmax= []
            for ii in range(len(file1[jj])):
                if z == 0:
                    locx.append(file1[jj][ii])
                    locx.append(file2[jj][ii])
                    z +=1
                elif z == 1:
                    locy.append(file1[jj][ii])
                    locy.append(file2[jj][ii])
                    z +=1                        
                elif z == 2:
                    loct.append(file1[jj][ii])
                    loct.append(file2[jj][ii])
                    z +=1                        

                elif z == 3:
                    locmin.append(((locx[-2]+locy[-2])/2 - (((locx[-2]-locy[-2])/2)**2 +loct[-2]**2)**(1/2)))
                    locmax.append(((locx[-2]+locy[-2])/2 + (((locx[-2]-locy[-2])/2)**2 +loct[-2]**2)**(1/2)))
                    
                    locmin.append(((locx[-1]+locy[-1])/2 - (((locx[-1]-locy[-1])/2)**2 +loct[-1]**2)**(1/2)))
                    locmax.append(((locx[-1]+locy[-1])/2 + (((locx[-1]-locy[-1])/2)**2 +loct[-1]**2)**(1/2)))                    
                    z += 1
                elif z == 4:
                    z = 0
                else:
                    z+=1      
            for ii in range(0,len(locx),int((self.nodes_along_width-1)*2)):
                y_stress.append(locx[ii:ii+int((self.nodes_along_width-1)*2)])
                x_stress.append(locy[ii:ii+int((self.nodes_along_width-1)*2)])
                tau_stress.append(loct[ii:ii+int((self.nodes_along_width-1)*2)])
                minP_stress.append(locmin[ii:ii+int((self.nodes_along_width-1)*2)])
                maxP_stress.append(locmax[ii:ii+int((self.nodes_along_width-1)*2)])
            sigx.append(y_stress)
            sigy.append(x_stress)
            tau.append(tau_stress)
            minP.append(minP_stress)
            maxP.append(maxP_stress)

        return tau,sigy,minP,maxP

    
    def plotMovie(self):
        
        tauxy,sigy,minPS,maxPS = self.extract(True)
        Epsxy, Epsy, minPE, maxPE = self.extract(False)
        
        
        x_loc, y_loc = self.getLocations()
        titles = ['Maximum Principal Strain' ,'Minimum Principal Strain' ,'Maximum Principal Stress','Minimum Principal Stress','Epsxy','Epsy','Tauxy','Sigy']
        
        fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.15)
        itr_1 = [maxPE[0] ,minPE[0],maxPS[0],minPS[0],Epsxy[0],Epsy[0] ,tauxy[0],sigy[0] ]
        itr_tot = [maxPE ,minPE,maxPS,minPS,Epsxy,Epsy,tauxy,sigy ]
    
        z_min = []
        for ii in range(len(itr_tot)):
            locmin = 100
            x = itr_tot[ii]
            for kk in range(len(itr_tot[ii])):
                x = itr_tot[ii][kk]
                for jj in range(len(x)):
                    xmin = min(x[jj])
                    if xmin<locmin:
                        locmin = xmin
            if locmin < -8:
                locmin = -8
            z_min.append(locmin)
                        
        z_max = []
        for ii in range(len(itr_tot)):
            locmax = 0
            x = itr_tot[ii]
            for kk in range(len(itr_tot[ii])):
                x = itr_tot[ii][kk]
                for jj in range(len(x)):
                    xmax = max(x[jj])
                    if xmax>locmax:
                        locmax = xmax
            if locmax > 1:
                locmax = 1
            z_max.append(locmax)
            
            
        for ii in range(int(2*len(itr_1))):
            if ii < len(itr_1):
                column = 1
                loc = ii
                cbx = 0.425
            else:
                loc = ii - len(itr_1)
                column = 2
                cbx = 1
            if ii == (len(itr_1) - 1) or ii == (2*len(itr_1) - 1) :
                showColorScale = True
            else:
                showColorScale = False
                
            fig.add_trace(go.Heatmap(z=itr_1[loc] ,x = x_loc, y = y_loc,    zmin=z_min[loc], zmax=z_max[loc],colorbar_x=cbx,showscale = showColorScale ), row=1, col=column)


        # Define frames
        number_frames = len(sigy)
        frames = [dict(
            name=k,
            
            data=[go.Heatmap(z=maxPE[k] ,zsmooth = False),
                  go.Heatmap(z=minPE[k],zsmooth = False),
                  go.Heatmap(z=maxPS[k],zsmooth = False),
                  go.Heatmap(z=minPS[k],zsmooth = False),
                  go.Heatmap(z=Epsxy[k],zsmooth = False),
                  go.Heatmap(z=Epsy[k] ,zsmooth = False),
                  go.Heatmap(z=tauxy[k],zsmooth = False),
                  go.Heatmap(z=sigy[k] ,zsmooth = False),
                  
                  go.Heatmap(z=maxPE[k] ,zsmooth = False),
                  go.Heatmap(z=minPE[k],zsmooth = False),
                  go.Heatmap(z=maxPS[k],zsmooth = False),
                  go.Heatmap(z=minPS[k],zsmooth = False),
                  go.Heatmap(z=Epsxy[k],zsmooth = False),
                  go.Heatmap(z=Epsy[k] ,zsmooth = False),
                  go.Heatmap(z=tauxy[k],zsmooth = False),
                  go.Heatmap(z=sigy[k] ,zsmooth = False)
                  ], 
        ) for k in range(number_frames)]



        button = []

        button.append(dict(label='Play',
                                          method='animate',
                                          args=[[f'{k}' for k in range(number_frames)],
                                                dict(frame=dict(duration=1, redraw=True),
                                                     transition=dict(duration=1),
                                                     easing='linear',
                                                     fromcurrent=True,
                                                     mode='immediate'
                                                     )]))
        button.append(dict(label='Pause',
                                          method='animate',
                                          args=[[None],
                                                dict(frame=dict(duration=0, redraw=False),
                                                     transition=dict(duration=0),
                                                     mode='immediate'
                                                     )]))



        drop_button = []
        drops = []
        updatemenus = []
        for ii in range(len(titles)):
            vis = [False,False,False,False,False,False,False,False]
            vis[-ii-1] = True
            loc_but = dict(args = [{'visible':vis,'showscale': vis},[0,1,2,3,4,5,6,7]],
                     label = titles[-ii-1],
                     method = 'restyle'
            )
            drop_button.append(loc_but)
        
        drop1 = dict(
            buttons = drop_button,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0,
            xanchor="left",
            y=1.25,
            yanchor="top")
        
        updatemenus.append(drop1)
        
        drop_button = []
        for ii in range(len(titles)):
            vis = [False,False,False,False,False,False,False,False]
            vis[-ii-1] = True
            loc_but = dict(args = [{'visible':vis,'showscale': vis},[8,9,10,11,12,13,14,15]],
                     label = titles[-ii-1],
                     method = 'restyle'
            )
            drop_button.append(loc_but)
        
        drop2 = dict(
            buttons = drop_button,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=.58,
            xanchor="left",
            y=1.25,
            yanchor="top")
                
        
        updatemenus.append(drop2)
    
        updatemenus.append(    dict(type='buttons',
                            buttons = button,
                            direction='left',
                            pad=dict(r=10, t=85),
                            showactive=True, x=0.1, y=0, xanchor='right', yanchor='top'))




        # Slider
        disp = []
        for ii in range(0,len(self.disp),self.skip):
            disp.append(self.disp[ii])
        sliders = [{'yanchor': 'top',
                    'xanchor': 'left',
                    'currentvalue': {'font': {'size': 16}, 'prefix': 'Drift: ', 'visible': True, 'xanchor': 'right'},
                    'transition': {'duration': 0, 'easing': 'linear'},
                    'pad': {'b': 10, 't': 50},
                    'len': 0.9, 'x': 0.1, 'y': 0,
                    'steps': [{'args': [[k], {'frame': {'duration': 1, 'easing': 'linear', 'redraw': True},
                                              'transition': {'duration': 0, 'easing': 'linear'}}],
                               'label': f"{round(disp[k],2)} %", 'method': 'animate'} for k in range(0,number_frames)
                              ]}]


        fig.update(frames=frames),
        fig.update_layout(height=800, width=1000,updatemenus=updatemenus,
                          sliders=sliders)


        #fig.show() 
        fig.write_html(self.filepath+"movie.html")
        pio.show(fig)   
        
    def finish(self):
        self.dispForce()
        self.files = [self.ip1filesig,self.ip2filesig,self.ip1fileeps,self.ip2fileeps]

        for ii in range(len(self.files)):
            loc = self.files[ii]
            loc = loc[::self.skip]
            self.files[ii] = loc
        minLoc = self.total
        for ii in range(len(self.files)):
            if len(self.files[ii])< minLoc:
                minLoc = len(self.files[ii])
        for ii in range(len(self.files)):
            loc = self.files[ii]
            loc = loc[0:minLoc]
            self.files[ii] = loc
                    
        self.num_itr = minLoc
        
        self.plotMovie()
        

In [ ]:
from ipynb.fs.full.XMLRead import *

In [ ]:
files = [
    'tcl_files//tcl_files_23030421//SeguraWP7_230304211618//230304211618referenceFile.txt',
     'tcl_files//tcl_files_23030420//LowesPW4_230304205135//230304205135referenceFile.txt',
       'tcl_files//tcl_files_23022600//OhWR20_230226004714//230226004714referenceFile.txt',
 'tcl_files//tcl_files_23030120//ZhangWangSW8_230301204146//230301204146referenceFile.txt'
]
for ii in files:
    xmlFiles = XMLRead(ii)
    cool = CreateMovies(ii,xmlFiles.ip1filesig,xmlFiles.ip2filesig,xmlFiles.ip1fileeps,xmlFiles.ip2fileeps)